In [1]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import numpy as np 
import torch_scatter.scatter as scatter 
from torch_geometric.utils import degree,softmax
from sklearn.metrics import roc_auc_score
from copy import deepcopy 
from collections import defaultdict

### Graph数据结构（存储方式）
对于图数据我们首先为图数据定义一种存储形式。下面是一种常见的定义方式：
- 节点属性 x：形状为(num_nodes,feature_dim)的矩阵
- 图结构 edge_index：以COO格式存储graph的邻接矩阵，形状为(2,num_edges)，第一行为源节点，第二行为目的节点

In [2]:
class Graph:
    def __init__(self,feat,edge_index) -> None:
        self.x = torch.tensor(feat)
        self.edge_index = torch.tensor(edge_index)

### GNN框架
GNN模型的实现关键在于实现GNN的Message Passing，即通过对节点和边的特征进行聚合来更新节点的特征。
Message Passing 可以分为三步：
1. 消息构建（Message）：对于每个节点，根据邻居节点的特征计算出一条消息；
2. 聚合（Aggregate）：将所有接收到的消息进行聚合
3. 更新（Update）：根据聚合后的信息,和节点本身的信息更新节点的特征

下面是一个简单的GNN框架，我们可基于这个框架实现GCN、GAT、GraphSage等GNN模型。
更加全面的框架可以参考PyG和DGL，我们只需要实现框架中的message，aggregate，update三个函数就能够完成一个GNN layer，同时我们给出了基于这个框架实现的GCN，GAT和GraphSAGE


In [3]:
class MessagePassing(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self,x,edge_index):
        out = self.propagate(x,edge_index)
        return out

    def propagate(self, x, edge_index):
        msg = self.message(x,edge_index)
        agg = self.aggregate(msg,edge_index)
        out = self.update(x,agg,edge_index)
        return out

    def message(self,x,edge_index):
        '''
        构建需要传递的消息
        '''
        src,dst = edge_index
        return x[dst]

    def aggregate(self,msg,edge_index):
        
        '''
        汇集来自邻居的消息
        '''
        src,dst = edge_index
        out = scatter(msg, src, dim=0, reduce='sum') # 聚合邻居节点的信息 
        return out
    
    def update(self,x,agg,edge_index):
        '''
        更新节点特征
        '''
        return agg



###  GCN
- GCN的消息传递机制为 $\mathbf{x}_i^{(k)}=\sum_{j\in\mathcal{N}(i)\cup\{i\}}\frac1{\sqrt{\deg(i)}\cdot\sqrt{\deg(j)}}\cdot\left(\mathbf{W}^\top\cdot\mathbf{x}_j^{(k-1)}\right)+\mathbf{b}$
- message：对邻居特征进行线性变换后归一化$\frac1{\sqrt{\deg(i)}\cdot\sqrt{\deg(j)}}\cdot\left(\mathbf{W}^\top\cdot\mathbf{x}_j^{(k-1)}\right)$
- aggregate：使用SumPooling汇聚邻居消息
- update：加上偏置项

In [4]:
class GCNConv(MessagePassing):
    def __init__(self,in_dim,out_dim):
        super().__init__()
        self.linear = nn.Linear(in_dim,out_dim,bias=False)
        self.bias = nn.Parameter(torch.zeros(out_dim).float())

    def message(self,x,edge_index):
        '''
        构建需要传递的消息
        '''
        x = self.linear(x) # 对邻居节点特征进行特征变换
        src,dst = edge_index  
        node_degree = degree(src)  
        norm = 1/torch.sqrt(node_degree[src]*node_degree[dst])
        norm[norm==float('inf')]=1
        msg = x[dst] * norm.view(-1,1)
        return msg

    def update(self, x, agg, edge_index):
        return agg + self.bias


### GAT
- GAT的消息传递机制为：$\vec{h}_i^{\prime}=\sigma\left(\sum_{j\in\mathcal{N}_1}\alpha_{ij}\mathbf{W}\vec{h}_j\right)$
- 其中$\alpha_{i j}=\operatorname{softmax}_{j}\left(e_{i j}\right)=\frac{\exp \left(e_{i j}\right)}{\sum_{k \in N_{i}} \exp \left(e_{i k}\right)}=\frac{\exp \left(\operatorname{LeakyReLU}\left(\overrightarrow{\mathbf{a}}^{T}\left[\mathbf{W} \vec{h}_{i} \| \mathbf{W} \vec{h}_{j}\right]\right)\right)}{\sum_{k \in \mathcal{N}_{i}} \exp \left(\operatorname{LeakyReLU}\left(\overrightarrow{\mathbf{a}}^{T}\left[\mathbf{W} \vec{h}_{i} \| \mathbf{W} \vec{h}_{k}\right]\right)\right)}$
- message:带attention权重的消息
- aggregate：Sumpooling

In [5]:
class GATConv(MessagePassing):
    def __init__(self,in_dim,out_dim):
        super().__init__()
        self.out_dim = out_dim
        self.linear = nn.Linear(in_dim, out_dim)
        self.a = nn.Parameter(torch.rand(size=(2*out_dim, 1)))
        self.leakyrelu = nn.LeakyReLU()
    
    def message(self, x, edge_index):
        Wh = self.linear(x)
        Wh1 = torch.matmul(Wh, self.a[:self.out_dim, :])
        Wh2 = torch.matmul(Wh, self.a[self.out_dim:, :])
        src,dst = edge_index
        e = self.leakyrelu(Wh1[src] + Wh2[dst])
        norm = softmax(e,index=src)
        return norm.view(-1,1) * Wh[dst]


### GraphSAGE

- 消息传递机制为$h_v^{(l)} = W_l\cdot h_v^{(l-1)} + W_r \cdot AGG(\{h_u^{(l-1)}, \forall u \in N(v) \})$
- message：原始的属性
- aggregate：根据原始论文，可以为SumPooling，MeanPooing，MaxPooling等
- update：分别处理节点自身消息和邻居后求和


In [6]:
class SAGEConv(MessagePassing):
    def __init__(self,in_dim,out_dim,pooling = 'sum'):
        super().__init__()
        self.linear_l = nn.Linear(in_dim,out_dim)
        self.linear_r = nn.Linear(in_dim,out_dim) 
        self.pooling = pooling
    def aggregate(self, msg, edge_index):
        src,dst = edge_index
        if self.pooling == 'sum':
            agg = scatter(msg, src, dim=0, reduce='sum') 
        elif self.pooling == 'mean':
            agg = scatter(msg, src, dim=0, reduce='mean') 
        elif self.pooling == 'max':
            agg = scatter(msg, src, dim=0, reduce='max') 
        return agg 
    
    def update(self, x, agg, edge_index):
        out = self.linear_l(x) + self.linear_r(agg) 
        return out 

## 节点分类任务
- 节点分类任务是针对图数据的节点进行分类。通常具有transductive和inductive两种设置。其中transductive设置下，模型对所有节点均可见，而inductive设置下，模型仅对训练集上的节点可见。
- 节点分类任务的输入为节点的特征向量和图的结构，输出为节点的类别标签。
- 节点分类任务的评价指标有准确率、F1值等。

#### 1、载入数据集以Cora数据集为例
- 利用pyg的api，将cora数据集加载到内存中


In [7]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='data', name='Cora')

In [8]:
feat = dataset.data.x 
label = dataset.data.y
edge_index = dataset.data.edge_index
# 使用预定义好的数据集划分
train_mask = dataset.data.train_mask
val_mask = dataset.data.val_mask
test_mask = dataset.data.test_mask

/Users/zhousheng/anaconda3/lib/python3.11/site-packages/torch_geometric/data/in_memory_dataset.py:284: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


#### 2、构建GNN模型
- 利用我们之前实现的MessagePassing搭建GNN模型
- 以GCN为例：

In [9]:
class GNN(nn.Module):
    def __init__(self,in_dim,hid_dim,out_dim,num_layers=2,dropout=0.1):
        super().__init__()
        self.layers = nn.ModuleList()
        self.layers.append(GCNConv(in_dim,hid_dim))
        for _ in range(num_layers-2):
            self.layers.append(GCNConv(hid_dim,hid_dim))
        self.layers.append(GCNConv(hid_dim,out_dim))
        self.act = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
    
    def forward(self,x,edge_index):
        h = x 
        for layer in self.layers[:-1]:
            h = self.act(layer(h,edge_index))
            h = self.dropout(h) 
        h = self.layers[-1](h,edge_index)         
        return h

model = GNN(feat.shape[1],64,max(label)+1,num_layers=3)

#### 3、训练模型
- 节点分类常用的训练损失为CrossEntropyLoss

In [10]:
# 定义优化器
optim = torch.optim.Adam(model.parameters(),lr = 0.01)
# 定义损失函数
lossfunc = nn.CrossEntropyLoss()
# 训练模型
best_val_acc = 0 # 记录验证集的最好结果
best_param = None # 记录最佳参数

for epoch in range(100): # 训练100个epoch
    model.train()
    predict = model(feat,edge_index)
    loss = lossfunc(predict[train_mask],label[train_mask])# 使用训练集计算损失
    optim.zero_grad()
    loss.backward()
    optim.step()
    # 在验证集上进行评估
    with torch.no_grad():
        model.eval()
        val_predict = model(feat,edge_index)[val_mask]
        # 计算准确率
        acc =  (val_predict.argmax(-1) == label[val_mask]).float().mean()
        if acc > best_val_acc:
            best_val_acc = acc
            best_param =  deepcopy(model.state_dict())
    print('Epoch %d, Loss %.4f, Val Accuracy %.4f'%(epoch,loss,acc))

Epoch 0, Loss 1.9454, Val Accuracy 0.3900
Epoch 1, Loss 1.9081, Val Accuracy 0.4580
Epoch 2, Loss 1.8070, Val Accuracy 0.4900
Epoch 3, Loss 1.6526, Val Accuracy 0.4820
Epoch 4, Loss 1.4614, Val Accuracy 0.5300
Epoch 5, Loss 1.2138, Val Accuracy 0.5980
Epoch 6, Loss 0.9608, Val Accuracy 0.6880
Epoch 7, Loss 0.7208, Val Accuracy 0.7220
Epoch 8, Loss 0.5187, Val Accuracy 0.7700
Epoch 9, Loss 0.3539, Val Accuracy 0.7820
Epoch 10, Loss 0.2267, Val Accuracy 0.7840
Epoch 11, Loss 0.1444, Val Accuracy 0.7740
Epoch 12, Loss 0.0985, Val Accuracy 0.7760
Epoch 13, Loss 0.0757, Val Accuracy 0.7660
Epoch 14, Loss 0.0453, Val Accuracy 0.7500
Epoch 15, Loss 0.0336, Val Accuracy 0.7520
Epoch 16, Loss 0.0192, Val Accuracy 0.7620
Epoch 17, Loss 0.0179, Val Accuracy 0.7580
Epoch 18, Loss 0.0104, Val Accuracy 0.7540
Epoch 19, Loss 0.0062, Val Accuracy 0.7540
Epoch 20, Loss 0.0029, Val Accuracy 0.7500
Epoch 21, Loss 0.0023, Val Accuracy 0.7400
Epoch 22, Loss 0.0013, Val Accuracy 0.7380
Epoch 23, Loss 0.0011

#### 4、预测结果

In [11]:
# 载入最优模型
model.load_state_dict(best_param)
# 测试集评估
with torch.no_grad():
    model.eval()
    test_predict = model(feat,edge_index)[test_mask]
    # 计算准确率
    acc =  (test_predict.argmax(-1) == label[test_mask]).float().mean()
print('Test Accuracy: {:.4f}'.format(acc))

Test Accuracy: 0.7750


## 链接预测任务
- 链接预测任务是图数据分析中的一个重要任务，旨在预测图中节点之间是否存在链接。换句话说，它试图预测图中尚未观察到的潜在边缘，或者评估现有边缘的可能性。链接预测在多种应用中都非常重要，如社交网络分析、生物信息学、推荐系统等。
- 链接预测任务的输入为节点的特征向量和图的结构，输出为任意两个节点间存在边的概率
- 链接预测的评估指标有AUC，Recall，Precision等

#### 载入数据集
  将图上的链接划分为训练集(0.6)，验证集(0.2)，和测试集(0.2),其中训练集的80%用于消息传递，20%用于计算loss训练。

In [12]:
# 读取数据集
dataset = Planetoid(root='data', name='Cora')
feat = dataset.data.x 
edge_index = dataset.data.edge_index

# 划分数据集
num_edges = edge_index.shape[1]
rand_index = torch.randperm(num_edges)
edge_index = edge_index[:,rand_index]

train_num = int(0.6*num_edges)
val_num = int(0.2*num_edges) 

train_edge = edge_index[:,:train_num]
val_edge_label_index = edge_index[:,train_num:train_num+val_num]
test_edge_label_index = edge_index[:,train_num+val_num:]

index_num = int(0.8*train_num)
train_edge_index = train_edge[:,:index_num]
train_edge_label_index = train_edge[:,index_num:]

- 链接预测任务可以视为一个二分类任务，然而由于graph的稀疏特性，具有大量的负样本，使用全部负样本进行训练不太现实，因此我们需要采样部分负样本进行训练和评估。

In [13]:
# 负采样
pos_edge_dict = defaultdict(set)
for i in range(train_edge.shape[1]):
    pos_edge_dict[train_edge[0,i]].add(train_edge[1,i])
    pos_edge_dict[train_edge[1,i]].add(train_edge[0,i])

all_pos_dict = defaultdict(set)
for i in range(train_edge.shape[1]):
    pos_edge_dict[train_edge[0,i]].add(train_edge[1,i])
    pos_edge_dict[train_edge[1,i]].add(train_edge[0,i])

def negative_sample(pos_edge_dict,num_nodes,num_neg_samples=100):
    negative_src = np.random.choice(num_nodes, num_neg_samples)
    negative_dst = np.random.choice(num_nodes, num_neg_samples)
    for i in range(num_neg_samples):
        while negative_dst[i] in pos_edge_dict[negative_src[i]]:
            negative_dst[i] = np.random.choice(num_nodes)
    return torch.tensor([negative_src.tolist(),negative_dst.tolist()])

def get_data(pos_edge,pos_edge_dict):
    negative_edge = negative_sample(pos_edge_dict,pos_edge.shape[1])
    pos_label = torch.ones_like(pos_edge[0])
    neg_label = torch.zeros_like(negative_edge[0])
    edge_label_index = torch.cat((pos_edge,negative_edge),axis=-1)
    edge_label =  torch.cat((pos_label,neg_label)).float()
    return  edge_label_index,edge_label


In [14]:
# 为验证集和测试集进行固定负采样用于评估
val_edge_label_index,val_edge_label = get_data(val_edge_label_index,all_pos_dict)
test_edge_label_index,test_edge_label = get_data(test_edge_label_index,all_pos_dict)

#### 构建模型
- 不同于节点分类直接使用GNN输出节点类别，链接预测中通常将GNN作为编码器来获取节点的表征，再利用解码器来建模两节点间的连边概率。
- 解码器通常为直接计算节点表征的dot product，也可以将节点表征拼接后使用mlp预测。


In [15]:
class LinkPredict(nn.Module):
    def __init__(self,in_dim,hid_dim,num_layers=2,dropout=0.1):
        super().__init__()
        self.encoder = GNN(in_dim,hid_dim,hid_dim,num_layers=num_layers,dropout=dropout)
        self.act = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
    
    def forward(self,x,edge_index,predict_edge):
        node_rep = self.encoder(x,edge_index)
        prob = (node_rep[predict_edge[0]] * node_rep[predict_edge[1]]).sum(dim=1)     
        return prob 

model = LinkPredict(feat.shape[1],64,num_layers=3)

#### 模型训练

In [16]:
# 定义优化器
optim = torch.optim.Adam(model.parameters(),lr = 0.01)
# 定义损失函数
lossfunc = nn.BCEWithLogitsLoss()
# 训练模型
best_val_auc = 0 # 记录验证集的最好结果
best_param = None # 记录最佳参数

for epoch in range(100): # 训练100个epoch
    model.train()
    edge_label_index,edge_label = get_data(train_edge_label_index,pos_edge_dict)
    predict = model(feat,train_edge_index,edge_label_index)
    loss = lossfunc(predict,edge_label)
    optim.zero_grad()
    loss.backward()
    optim.step()

    # 在验证集上进行评估
    with torch.no_grad():
        model.eval()
        predict = model(feat,train_edge_index,val_edge_label_index)
        auc = roc_auc_score(val_edge_label.long().numpy(),predict.numpy())
        if auc > best_val_auc:
            best_val_auc = auc
            best_param =  deepcopy(model.state_dict())
    print('Epoch %d, Loss %.4f, Val AUC %.4f'%(epoch,loss,auc))

Epoch 0, Loss 0.6923, Val AUC 0.6352
Epoch 1, Loss 0.3593, Val AUC 0.6351
Epoch 2, Loss 0.7134, Val AUC 0.6401
Epoch 3, Loss 0.3675, Val AUC 0.6544
Epoch 4, Loss 0.4395, Val AUC 0.6648
Epoch 5, Loss 0.5163, Val AUC 0.6677
Epoch 6, Loss 0.4819, Val AUC 0.6662
Epoch 7, Loss 0.3931, Val AUC 0.6627
Epoch 8, Loss 0.3005, Val AUC 0.6591
Epoch 9, Loss 0.2998, Val AUC 0.6574
Epoch 10, Loss 0.3543, Val AUC 0.6584
Epoch 11, Loss 0.3821, Val AUC 0.6620
Epoch 12, Loss 0.2965, Val AUC 0.6662
Epoch 13, Loss 0.2753, Val AUC 0.6714
Epoch 14, Loss 0.2687, Val AUC 0.6768
Epoch 15, Loss 0.2613, Val AUC 0.6809
Epoch 16, Loss 0.2672, Val AUC 0.6844
Epoch 17, Loss 0.2713, Val AUC 0.6871
Epoch 18, Loss 0.2676, Val AUC 0.6897
Epoch 19, Loss 0.2569, Val AUC 0.6910
Epoch 20, Loss 0.2524, Val AUC 0.6918
Epoch 21, Loss 0.2413, Val AUC 0.6926
Epoch 22, Loss 0.2385, Val AUC 0.6933
Epoch 23, Loss 0.2455, Val AUC 0.6946
Epoch 24, Loss 0.2668, Val AUC 0.6963
Epoch 25, Loss 0.2471, Val AUC 0.6980
Epoch 26, Loss 0.2428,

#### 在测试集上评估

In [17]:
# 载入最优模型
model.load_state_dict(best_param)
# 测试集评估
with torch.no_grad():
    model.eval()
    predict = model(feat,train_edge_index,test_edge_label_index)
    auc = roc_auc_score(test_edge_label.long().numpy(),predict.numpy())
print('Test AUC: {:.4f}'.format(auc))

Test AUC: 0.6643
